In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.dates as mdates  #Importing module for date formatting
import matplotlib.ticker as ticker  # Importing ticker for customizing colorbar ticks
from scipy.ndimage import gaussian_filter
import seaborn as sns
from metpy.calc import virtual_potential_temperature
from scipy import signal
import glob
from datetime import datetime
import os

from matplotlib.colors import Normalize
import matplotlib.colors as mcolors

In [2]:
ds = xr.open_dataset('./all-les-variables.nc')
ds['time'] = pd.to_datetime(ds['time'], unit='s')
ds

<xarray.Dataset>
Dimensions:                    (Time: 12520, bottom_top: 290, layers: 10,
                                time: 12520, h: 290)
Coordinates:
  * Time                       (Time) datetime64[ns] 2020-03-13T08:21:00 ... ...
  * time                       (time) datetime64[ns] 2020-03-13T08:21:00 ... ...
Dimensions without coordinates: bottom_top, layers, h
Data variables: (12/35)
    XLONG                      (Time) float32 ...
    XLAT                       (Time) float32 ...
    H                          (Time, bottom_top) float32 ...
    Base_Time                  (Time) int32 ...
    U                          (Time, bottom_top) float32 ...
    V                          (Time, bottom_top) float32 ...
    ...                         ...
    ARSCL_CB                   (Time, layers) float32 ...
    temp-anomaly               (Time) float32 ...
    hydrometeors-mixing-ratio  (Time, bottom_top) float32 ...
    precipitation              (Time) float32 ...
    height                     (bottom_top) float32 ...
    cloud_mask                 (time, h) int64 ...

In [3]:
ds

<xarray.Dataset>
Dimensions:                    (Time: 12520, bottom_top: 290, layers: 10,
                                time: 12520, h: 290)
Coordinates:
  * Time                       (Time) datetime64[ns] 2020-03-13T08:21:00 ... ...
  * time                       (time) datetime64[ns] 2020-03-13T08:21:00 ... ...
Dimensions without coordinates: bottom_top, layers, h
Data variables: (12/35)
    XLONG                      (Time) float32 15.69 15.69 15.69 ... 15.69 15.69
    XLAT                       (Time) float32 69.14 69.14 69.14 ... 69.14 69.14
    H                          (Time, bottom_top) float32 ...
    Base_Time                  (Time) int32 1584087660 1584087664 ... 1584143995
    U                          (Time, bottom_top) float32 ...
    V                          (Time, bottom_top) float32 ...
    ...                         ...
    ARSCL_CB                   (Time, layers) float32 ...
    temp-anomaly               (Time) float32 -0.6404 -0.6385 ... -0.5864
    hydrometeors-mixing-ratio  (Time, bottom_top) float32 ...
    precipitation              (Time) float32 0.0 0.0 0.0 ... 0.272 0.264 0.256
    height                     (bottom_top) float32 160.0 190.0 ... 8.831e+03
    cloud_mask                 (time, h) int64 ...

In [4]:
time = ds['Time']
height = ds['H'].mean('Time')
hydrometeors_qr = ds['hydrometeors-mixing-ratio']
vertical_velocity = ds['W']
tke = ds['TKE']
lwp = ds['LWP']
precip = ds['precipitation']
temp_anomaly = ds['temp-anomaly']
mask_cloud = ds['cloud_mask']
U, V = ds['U'].mean('Time'), ds['V'].mean('Time')

In [5]:
# speeds = np.sqrt(U.values, V.values)
# len(speeds)

In [6]:
# # Time values, in this case just a sample, you'll replace this with your time array
# times = np.array(['2020-03-13T12:00:00.000000000', '2020-03-13T12:00:04.000000000', 
#                   '2020-03-13T12:00:09.000000000', '2020-03-13T12:00:59.000000000', 
#                   '2020-03-13T12:01:59.551000000', '2020-03-13T12:02:59.550000000'], 
#                  dtype='datetime64[ns]')

# # Convert the times to seconds relative to the first timestamp
# relative_seconds = (times - times[0]).astype('timedelta64[s]').astype(int)


In [7]:
# len(time.values)

In [8]:
# # Let's assume 'times' is your array of datetime64[ns] timestamps
# # Example array, replace with your actual data:
# times = time.values

# # Convert the times to seconds relative to the first timestamp
# relative_seconds = (times - times[0]).astype('timedelta64[s]').astype(int)

# relative_seconds

In [9]:
# len(relative_seconds[:290])

In [10]:
# speeds * relative_seconds[:290

In [2]:
def vertical_transect(formatted_date:list, CAO_date:str):
    # open the dataset
    ds = xr.open_dataset('./all-les-variables.nc')
    ds['time'] = pd.to_datetime(ds['time'], unit='s')
    
    ds_subsets = []; ds_subsets1 = []; heights = [];
    hydrometeors_qrs = []; vertical_velocitys = []; tkes = []; lwps = []; precips = []; temp_anomalys = []; mask_clouds = [];
   

    for entry in formatted_date:
        name, start_time, end_time = entry
        ds_subset = ds.sel(Time=slice(start_time, end_time))
        ds_subset1 = ds.sel(time=slice(start_time, end_time))
        # exteact needed dataset
        time = ds_subset['Time']
        height = ds_subset['H'].mean('Time')
        hydrometeors_qr = ds_subset['hydrometeors-mixing-ratio']
        vertical_velocity = ds_subset['W']
        tke = ds_subset['TKE']
        lwp = ds_subset['LWP']
        precip = ds_subset['precipitation']
        temp_anomaly = ds_subset['temp-anomaly']
        mask_cloud = ds_subset1['cloud_mask']
      
        # append new results(variables)
        ds_subsets.append(ds_subset)
        ds_subsets1.append(ds_subset1)
        heights.append(height)

        hydrometeors_qrs.append(hydrometeors_qr)
        vertical_velocitys.append(vertical_velocity)
        tkes.append(tke)
        lwps.append(lwp)
        precips.append(precip)
        temp_anomalys.append(temp_anomaly)
        mask_clouds.append(mask_cloud)
 
    
    # for j in range(0, len(formatted_date)):
        
    for j, k in enumerate(formatted_date):
        
        # plotting function
        fig, ax = plt.subplots(figsize=(8, 15), nrows=6, ncols=1, sharex=True, constrained_layout=True)
        ax = ax.flatten()


        # Generate the mask based on hydrometeors
        hydrometeor_data = ds_subsets[j]['hydrometeors-mixing-ratio'].T.values
        threshold = np.nanmean(hydrometeor_data)  # Or some other threshold
        mask = hydrometeor_data > threshold

        # Apply the mask to the necessary variables
        masked_vertical_velocity = np.where(mask, ds_subsets[j]['W'].T.values, np.nan)
        masked_tke = np.where(mask, ds_subsets[j]['TKE'].T.values, np.nan)


      
        
        for i, v in enumerate(ax):
            labels = ["(a)", "(b)", "(c)", "(d)", "(e)", "(f)"]
            # Add text to the subplots
            ax[i].text(0.004, 0.97, labels[i], transform=ax[i].transAxes,
            fontsize=18, va='top', ha='left', fontweight='bold') 
            
            if i == 0:
                
                # add id labels
                ax[i].text(0.5, 1.1, f"ID-{k[0]}", fontsize=35, transform=ax[0].transAxes, ha='center')
                pcm = ax[i].pcolormesh(ds_subsets[j]['Time'], 
                                       ds_subsets[j]['height']/1000.0, 
                                       ds_subsets[j]['hydrometeors-mixing-ratio'].T, 
                                       cmap='nipy_spectral', vmin=-8, vmax=-2, zorder=0)

                ax[i].set_ylim(0.6, 7)
                # Set y-axis ticks
                ax[i].set_yticks(np.arange(1, 8))
                ax[i].tick_params(axis='y', labelsize=15)
                ax[i].set_ylabel('Height [km]', color='black', fontsize=13)
                cbar = fig.colorbar(pcm, ax=ax[i], extend='neither')
                cbar.set_label('Hydrometeor\n Mixing Ratio[g/kg]', fontsize=11)
                # Adjust colorbar ticks for the specific range
                cbar.set_ticks(np.linspace(-8, -2, 7))  # Creates 7 evenly spaced ticks from -8 to -2
                cbar.set_ticklabels(['-8', '-7', '-6', '-5', '-4', '-3', '-2'])  # Optional: set specific tick labels
            if i == 1:
                pcm = ax[i].pcolormesh(ds_subsets[j]['Time'], 
                                       ds_subsets[j]['height']/1000.0, 
                                       masked_vertical_velocity, 
                                       cmap='bwr', vmin=-3, vmax=3, zorder=0)
                ax[i].set_ylim(0.6, 7)
                # Set y-axis ticks
                ax[i].set_yticks(np.arange(1, 8))
                ax[i].tick_params(axis='y', labelsize=15)
                ax[i].set_ylabel('Height [km]', color='black', fontsize=13)
                cbar = fig.colorbar(pcm, ax=ax[i], extend='both')
                cbar.set_label('Vertical Velocity [m s$^{-1}$]', fontsize=11)

            if i == 2:
                
                pcm = ax[i].pcolormesh(ds_subsets[j]['Time'], 
                                       ds_subsets[j]['height']/1000.0, 
                                       masked_tke, 
                                       cmap='Blues', vmin=0, vmax=1, zorder=0)
                ax[i].set_ylim(0.6, 7)
                # Set y-axis ticks
                ax[i].set_yticks(np.arange(1, 8))
                ax[i].tick_params(axis='y', labelsize=15)
                ax[i].set_ylabel('Height [km]', color='black', fontsize=13)
                cbar = fig.colorbar(pcm, ax=ax[i], extend='max')
                cbar.set_label('TKE [m$^{2}$ s$^{-2}$]', fontsize=13)

           
            if i == 3:
                # LWP plot setup
                pcm = ax[i].scatter(ds_subsets[j]['Time'], ds_subsets[j]['LWP'], color='blue', zorder=3, s=7)
                ax[i].spines['left'].set_color('blue')
                ax[i].tick_params(axis='y', colors='blue', labelsize=15)
                ax[i].set_ylabel("LWP [kg m$^{-2}$]", color="blue", fontsize=13)
                
                # Set y-axis limits for LWP and define ticks
                ax[i].set_ylim(0, 1.2)
                lwp_ticks = np.array([0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2])  # Explicitly define tick positions
                ax[i].set_yticks(lwp_ticks)
                # Format tick labels with one significant figure for non-integer values, ensuring correct display
                lwp_tick_labels = [f'{tick:.1f}' if tick % 1 else f'{int(tick)}' for tick in lwp_ticks]
                ax[i].set_yticklabels(lwp_tick_labels)
            
                # Invisible colorbar for LWP (maintaining previous setup)
                cbar = fig.colorbar(pcm, ax=ax[i], extend='neither')
                cbar.ax.set_visible(False)
                ax[i].grid(axis="y", linestyle='--', color="blue", alpha=0.5)
                            
                            # Twin y-axis for Precipitation
                ax31 = ax[i].twinx()
                ax31.scatter(ds_subsets[j]['Time'], ds_subsets[j]['precipitation'], color='red', s=2)
                ax31.spines['right'].set_color('red')
                ax31.tick_params(axis='y', colors='red', labelsize=15)
                ax31.set_ylabel('Precipitation [mm hr$^{-1}$]', color='red', fontsize=13)
                
                # Set y-axis limits for Precipitation
                precip_ticks = np.linspace(0, 5, 6)  # Creates ticks from 0 to 5, considering the max value and a little extra
                ax31.set_ylim(0, 5)  # Setting fixed upper limit slightly above max Precipitation
                ax31.set_yticks(precip_ticks)
                ax31.grid(axis="y", linestyle='--', color="red", alpha=0.5)
            

        
            if i == 4:
                pcm = ax[i].plot(ds_subsets[j]['Time'], ds_subsets[j]['temp-anomaly'], color='red')
                # pcm = ax[i].scatter(ds_subsets[j]['Time'], temp_anomalys[j], color='red', zorder=1, s=1)
                ax[i].axhline(y=0, color='black', linestyle='--', linewidth=2)
                ax[i].tick_params(axis='y', colors='black', labelsize=15)
    
                latex_part = r"$\theta_v - \overline{\theta_{v,6h}}\,[K]$"
                anomaly_part = "Temp. Anomaly"
                ax[i].set_ylabel(f"{anomaly_part}\n{latex_part}", fontsize=13, c='red')
                # Set y-axis limits for LWP and define ticks
                ax[i].set_ylim(-1.2, 1.7)
                anom_ticks = np.array([-1.2, -0.6,  0. ,  0.6,  1.2])  # Explicitly define tick positions
                ax[i].set_yticks(anom_ticks)
                # Format tick labels with one significant figure for non-integer values, ensuring correct display
                anom_tick_labels = [f'{tick:.1f}' if tick % 1 else f'{int(tick)}' for tick in anom_ticks]
                ax[i].set_yticklabels(anom_tick_labels)


          
            if i == 5:  
                from matplotlib.colors import ListedColormap
                cmap = ListedColormap(['red', 'blue', 'green'])
                

                # Set areas where there is no cloud to NaN
                cloud_mask_mod = ds_subsets1[j]['cloud_mask'].T.where(ds_subsets1[j]['cloud_mask'].T >= 1, np.nan)
            
                # Apply modulo operation to cycle colors within the range [1, 4]
                cloud_mask_mod = (cloud_mask_mod % 3) + 1  # Adjust the range to [1, 4]
            
                pcm = ax[i].pcolormesh(ds_subsets[j]['Time'],
                                       ds_subsets[j]['height']/1000.0,
                                       cloud_mask_mod,
                                       cmap=cmap, vmin=1, vmax=3)
                # Remove y-axis ticks from the current axis (ax[5])
                ax[5].set_yticks([])
        
            
                # ax[i].set_ylim(0.6, 7)
                # ax[i].set_yticks(np.arange(1, 8))
                # ax[i].tick_params(axis='y', labelsize=15)
                # ax[i].set_ylabel('Height [km]', color='black', fontsize=15)

        
            



 

     # =========================================================================================================
        # Custom datetime format
        custom_date_format = "%H:%M UTC\n%d %B %Y"  # Format: 04:00 UTC\n13 March 2020
        # Formatting the time labels
        ax[len(ax)-1].xaxis.set_major_formatter(mdates.DateFormatter(custom_date_format))
        # Including the first-date and last-date on the plot
        ax[len(ax)-1].set_xlim([ds_subsets[j]['Time'].min(), ds_subsets[j]['Time'].max()])
        # Set custom tick positions for the x-axis
        # num_ticks = 6  # Number of tick labels
        num_ticks = 5  # Number of tick labels
        tick_positions = np.linspace(0, len(ds_subsets[j]['Time']) - 1, num_ticks, dtype=int)
        ax[len(ax)-1].set_xticks(ds_subsets[j]['Time'][tick_positions])

        # Generate custom tick labels including the first and last dates
        tick_labels = [ds_subsets[j]['Time'][pos].dt.strftime(custom_date_format).values for pos in tick_positions]
        tick_labels[0] = ds_subsets[j]['Time'].min().dt.strftime(custom_date_format).values  # First date
        tick_labels[-1] = ds_subsets[j]['Time'].max().dt.strftime(custom_date_format).values  # Last date
        ax[len(ax)-1].set_xticklabels(tick_labels ,rotation=0);  # You can adjust fontsize and rotation
        
        
        # convert datetime from '%Y-%m-%dT%H:%M:%S.%f' to '%Y-%m-%d %H:%M:%S' format
        timestamp_start = str(ds_subsets[j]['Time'].values[0]) # start time
        timestamp_end = str(ds_subsets[j]['Time'].values[-1])  # end time
        
        timestamp_without_nanoseconds = timestamp_start[:-3]  # Remove last three digits
        timestamp_without_nanoseconds1 = timestamp_end[:-3]  # Remove last three digits
        
        timestamp_out_start = datetime.strptime(timestamp_without_nanoseconds, '%Y-%m-%dT%H:%M:%S.%f').strftime('%Y%m%d-%H:%M:%S')
        timestamp_out_end = datetime.strptime(timestamp_without_nanoseconds1, '%Y-%m-%dT%H:%M:%S.%f').strftime('%Y%m%d-%H:%M:%S')



        directory = f'LES-d02-single-cell-Open-cells-Figures/t_series/{CAO_date}'
        file_name = f"{timestamp_out_start}_{timestamp_out_end}_ID{k[0]}.png"


         # Create the directory if it doesn't exist
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(f"Directory '{directory}' created successfully.")

        dirr = sorted(os.listdir(directory))
        if file_name in dirr:
            # print(f'{dt[i]}.png already exist')
            pass
            plt.close()
        else:
            fig.savefig(os.path.join(directory, file_name), dpi=500)
            print(f"Figures in {file_name} executed!")
            plt.close()
    print(f'\U0001f600\U0001f600\U0001f600\U0001f600IDENTIFIED ALL {j+1} CELLS IDENTIFIED SUCCESSFULLY!!!\U0001f600\U0001f600\U0001f600\U0001f600')

In [3]:


# these start and end times are for the d02
data = [
    (1, '2020-03-13 08:25:21', '2020-03-13 08:35:19'),
    (2, '2020-03-13 08:37:07', '2020-03-13 09:07:34'),
    (3, '2020-03-13 09:06:09', '2020-03-13 09:23:19'),
    (4, '2020-03-13 09:38:19', '2020-03-13 10:15:22'),
    (5, '2020-03-13 10:09:27', '2020-03-13 10:22:03'),
    (6, '2020-03-13 10:16:16', '2020-03-13 10:37:07'),
    (7, '2020-03-13 10:34:12', '2020-03-13 11:02:28'),
    (8, '2020-03-13 10:52:07', '2020-03-13 11:01:52'),
    (9, '2020-03-13 10:59:19', '2020-03-13 11:11:06'),
    (10, '2020-03-13 11:08:33', '2020-03-13 11:30:22'),
    (11, '2020-03-13 11:24:54', '2020-03-13 11:45:45'),
    (12, '2020-03-13 11:46:48', '2020-03-13 11:59:33'),
#     (13, '2020-03-13 11:58:30', '2020-03-13 12:07:52'),
#     (14, '2020-03-13 12:05:24', '2020-03-13 12:34:43'),
#     (15, '2020-03-13 12:39:54', '2020-03-13 12:51:58'),
#     (16, '2020-03-13 12:50:51', '2020-03-13 13:20:33'),
#     (17, '2020-03-13 13:27:13', '2020-03-13 13:54:00'),
#     (18, '2020-03-13 13:39:31', '2020-03-13 13:55:34'),
#     (19, '2020-03-13 14:06:31', '2020-03-13 14:40:34'),
#     (20, '2020-03-13 14:40:16', '2020-03-13 15:00:22'),
#     (21, '2020-03-13 14:54:58', '2020-03-13 15:07:25'),
#     (22, '2020-03-13 15:05:28', '2020-03-13 15:19:52'),
#     (23, '2020-03-13 15:12:40', '2020-03-13 15:55:16'),
#     (24, '2020-03-13 15:50:42', '2020-03-13 16:09:36'),
#     (25, '2020-03-13 16:04:07', '2020-03-13 16:46:16'),
#     (26, '2020-03-13 16:43:52', '2020-03-13 16:59:42'),
#     (27, '2020-03-13 16:57:00', '2020-03-13 17:38:24'),
#     (28, '2020-03-13 17:05:10', '2020-03-13 17:24:13'),
#     (29, '2020-03-13 17:37:16', '2020-03-13 17:39:58'),
#     (30, '2020-03-13 17:33:09', '2020-03-13 18:01:34'),
#     (31, '2020-03-13 17:58:57', '2020-03-13 18:24:22'),
#     (32, '2020-03-13 18:20:46', '2020-03-13 18:52:34'),
#     (33, '2020-03-13 18:55:03', '2020-03-13 19:04:57'),
#     (34, '2020-03-13 19:03:04', '2020-03-13 19:23:10'),
#     (35, '2020-03-13 19:18:04', '2020-03-13 19:31:03'),
#     (36, '2020-03-13 19:27:00', '2020-03-13 19:55:16'),
#     (37, '2020-03-13 19:55:57', '2020-03-13 20:18:27'),
#     (38, '2020-03-13 20:16:30', '2020-03-13 20:29:01'),
#     (39, '2020-03-13 20:28:57', '2020-03-13 20:56:24'),
#     (40, '2020-03-13 20:49:34', '2020-03-13 21:07:39'),
#     (41, '2020-03-13 21:06:27', '2020-03-13 21:22:21'),
#     (42, '2020-03-13 21:18:45', '2020-03-13 21:40:21'),
#     (43, '2020-03-13 21:36:09', '2020-03-13 21:55:21'),
#     (44, '2020-03-13 21:52:16', '2020-03-13 22:03:27'),
#     (45, '2020-03-13 22:00:00', '2020-03-13 22:13:57'),
#     (46, '2020-03-13 22:09:36', '2020-03-13 22:21:04'),
#     (47, '2020-03-13 22:20:06', '2020-03-13 23:04:57'),
#     (48, '2020-03-13 23:03:40', '2020-03-13 23:14:10'),
#     (49, '2020-03-13 23:09:04', '2020-03-13 23:17:24'),
#     (50, '2020-03-13 23:16:07', '2020-03-13 23:26:42'),
#     (51, '2020-03-13 23:24:49', '2020-03-13 23:34:03'),
#     (46, '2020-03-13 23:31:52', '2020-03-13 23:59:55'),

]



formatted_date = []

for entry in data:
    entry_id, start_str, end_str = entry
    start_dt = datetime.strptime(start_str, '%Y-%m-%d %H:%M:%S')
    end_dt = datetime.strptime(end_str, '%Y-%m-%d %H:%M:%S')
    
    formatted_start = start_dt.strftime('%Y-%m-%dT%H:%M:%S.%f000')
    formatted_end = end_dt.strftime('%Y-%m-%dT%H:%M:%S.%f000')
    
    formatted_date.append((entry_id, formatted_start, formatted_end))

In [4]:
vertical_transect(formatted_date=formatted_date, CAO_date='Mar13')

Figures in 20200313-08:25:21_20200313-08:35:19_ID1.png executed!
Figures in 20200313-08:37:07_20200313-09:07:34_ID2.png executed!
Figures in 20200313-09:06:09_20200313-09:23:19_ID3.png executed!
Figures in 20200313-09:38:19_20200313-10:15:22_ID4.png executed!
Figures in 20200313-10:09:27_20200313-10:22:03_ID5.png executed!
Figures in 20200313-10:16:16_20200313-10:37:07_ID6.png executed!
Figures in 20200313-10:34:12_20200313-11:02:28_ID7.png executed!
Figures in 20200313-10:52:07_20200313-11:01:52_ID8.png executed!
Figures in 20200313-10:59:19_20200313-11:11:06_ID9.png executed!
Figures in 20200313-11:08:33_20200313-11:30:22_ID10.png executed!
Figures in 20200313-11:24:54_20200313-11:45:45_ID11.png executed!
Figures in 20200313-11:46:48_20200313-11:59:33_ID12.png executed!
😀😀😀😀IDENTIFIED ALL 12 CELLS IDENTIFIED SUCCESSFULLY!!!😀😀😀😀


In [14]:
pwd

'/glade/derecho/scratch/noteng/masters-research/comble/LES/LES-final'